In [3]:
def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class]==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)

def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:  #Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]

def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name  #Here we build our decision tree  #Get attribute with maximum information gain
    node = find_winner(df)  #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])  #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {} #We make loop to construct a tree by calling this function recursively. #In this we check if the subset is pure and stops if it is pure. 
    for value in attValue:
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Buy_Computer'],return_counts=True)                        
        if len(counts)==1:  #Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively               
    return tree


import pandas as pd
import numpy as np
eps = np.finfo(float).eps
from numpy import log2 as log
df = pd.read_csv('buy_computer.csv')
print("\n Given Buy Computer Data Set:\n\n",df)
tree= buildTree(df)
import pprint
pprint.pprint(tree)



 Given Buy Computer Data Set:

       Age    Education Income Marital_Status  Buy_Computer
0   36-55     Master's   High         Single             1
1   18-35  High School    Low         Single             0
2   36-55     Master's    Low         Single             1
3   18-35   Bachelor's   High         Single             0
4     <18  High School    Low         Single             1
5   18-35   Bachelor's   High        Married             0
6   36-55   Bachelor's    Low        Married             0
7     >55   Bachelor's   High         Single             1
8   36-55     Master's    Low        Married             0
9     >55     Master's    Low        Married             1
10  36-55     Master's   High         Single             1
11    >55     Master's   High         Single             1
12    <18  High School   High         Single             0
13  36-55     Master's    Low         Single             1
14  36-55  High School    Low         Single             1
15    <18  High School 

In [9]:
test={'Age':'36-55','Education':'High School','Income':'Low','Marital_Status':'Married'}
def func(test, tree, default=None):
    attribute = next(iter(tree)) 
    print(attribute) 
    if test[attribute] in tree[attribute].keys():
        print(tree[attribute].keys())
        print(test[attribute])
        result = tree[attribute][test[attribute]]
        if isinstance(result, dict):
            return func(test, result)
        else:
            return result
    else:
        return default
ans = func(test, tree)
print(ans)

Age
dict_keys(['18-35', '36-55', '<18', '>55'])
36-55
Marital_Status
dict_keys(['Married', 'Single'])
Married
0
